In [22]:
import wget

In [23]:
# url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"
# wget.download(url)

In [24]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [25]:
import minsearch

In [26]:
import json

In [27]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [28]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


Index the Document

In [29]:
index =minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]

)

In [30]:
index.fit(documents)


In [31]:
#q='the course has already started, can i still enroll?'

In [32]:
from openai import OpenAI

In [33]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")




In [34]:
client = OpenAI(api_key=api_key)

In [35]:
def search(query):
    boost={'question':3.0,'section':0.5}

    results=index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [36]:
def build_prompt(query,search_results):
    prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ databalse. Use only the facts from the CONTEXT when answering the QUESTION.


Question :{question}

CONTEXT:
{context}
""".strip()


    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt

In [37]:
def llm(prompt):
    response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)
    return response.choices[0].message.content

In [38]:
def rag(query):
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [39]:
rag("the course has already started. Can i still enroll?")

"Yes, you can still enroll in the course even after it has started. However, be mindful of the deadlines for turning in the final projects. It's advisable not to leave everything until the last minute."


Pull the latest elastic search

docker run -it --rm --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" elasticsearch:9.0.1

after run this :

curl http://localhost:9200

In [56]:
from elasticsearch import Elasticsearch

In [57]:
es_client=Elasticsearch('http://localhost:9200')

In [58]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
print(es.info())


{'name': 'e00e5de693b9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BDVnoyMpTIWH1fmZvPHQWQ', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


Index Settings

In [60]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course-questions"
es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [61]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [62]:
from tqdm.auto import tqdm

In [64]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

In [79]:
query='I just discovered the course. Can i still join it?'

In [80]:
def elastic_search (query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name,body=search_query)

    result_docs=[]
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [82]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [83]:
rag(query)

'Yes, you can still join the course even if you have just discovered it. You are eligible to submit the homeworks without registration. However, be mindful of the deadlines for turning in the final projects and try not to leave everything until the last minute.'

In [81]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta